In [11]:
from pandas_datareader import data
import datetime
# required modules for Bokeh
from bokeh.plotting import figure, show, output_file
from bokeh.models.annotations import Title


startDate=datetime.datetime(2021,1,2)
endDate=datetime.datetime(2021,1,29)

#object type = DataFrame => store data in a var 'df'
df=data.DataReader(name="BTC-USD",data_source="yahoo",start=startDate,end=endDate)


In [12]:
def inc_dec(close, open):
    if close > open:
        value="Increase"
    elif close < open:
        value="Decrease"
    else:
        valu="Equal"
    return value
# assign values builds a list in a new Status column
df["Status"]=[inc_dec(close,open) for close, open in zip(df.Close,df.Open)]

In [18]:
# avg between open and close price
df["Middle"]=(df.Open+df.Close)/2

df["Height"]=abs(df.Close-df.Open)

In [19]:
df # print dataframe.

,High,Low,Open,Close,Volume,Adj Close,Status,Middle,Height
Date,,,,,,,,,
2021-01-02,33155.117188,29091.181641,29376.455078,32127.267578,67865420765,32127.267578,Increase,30751.861328,2750.812500
2021-01-03,34608.558594,32052.316406,32129.408203,32782.023438,78665235202,32782.023438,Increase,32455.715820,652.615234
2021-01-04,33440.218750,28722.755859,32810.949219,31971.914062,81163475344,31971.914062,Decrease,32391.431641,839.035156
2021-01-05,34437.589844,30221.187500,31977.041016,33992.429688,67547324782,33992.429688,Increase,32984.735352,2015.388672
2021-01-06,36879.699219,33514.035156,34013.613281,36824.363281,75289433811,36824.363281,Increase,35418.988281,2810.750000
2021-01-07,40180.367188,36491.191406,36833.875000,39371.042969,84762141031,39371.042969,Increase,38102.458984,2537.167969
2021-01-08,41946.738281,36838.636719,39381.765625,40797.609375,88107519480,40797.609375,Increase,40089.687500,1415.843750
2021-01-09,41436.351562,38980.875000,40788.640625,40254.546875,61984162837,40254.546875,Decrease,40521.593750,534.093750
2021-01-10,41420.191406,35984.628906,40254.218750,38356.441406,79980747690,38356.441406,Decrease,39305.330078,1897.777344


In [8]:
# type(date_increase) # pandas.core.indexes.datetimes.DatetimeIndex

pandas.core.indexes.datetimes.DatetimeIndex

In [21]:
# Build candlestick chart using Bokeh rectangle glyphs

p=figure(x_axis_type='datetime', width=1000, height=300)
p.title.text = "Candlestick Chart"

### Hours to miliseconds
hours_12=12*60*60*1000

# four mandatory params for each candle:
# 1. x-coord of rect centre point
# 2. y-coord of rect centre point
# 3. width of rect
# 4. height of rect

# # Plot candle sticks where close is gt than open - stock price is up for session
p.rect(df.index[df.Status=="Increase"],df.Middle[df.Status=="Increase"], 
      hours_12, df.Height[df.Status=="Increase"], fill_color="green", line_color="black") # abs ensures a positive value always

# Plot candle sticks where close is lt open - stock price is down for session
p.rect(df.index[df.Status=="Decrease"],df.Middle[df.Status=="Decrease"], hours_12, 
       df.Height[df.Status=="Decrease"], fill_color="red", line_color="black") # abs ensures a positive value always


output_file("CS.html")
show(p)
